# PyMC3 implementation of the model behind https://rt.live

Notebook by Maciek Zdanowicz & Grzegorz Kossakowski (@gkossakoski)

In [1]:
!curl -o euData0420.csv https://opendata.ecdc.europa.eu/covid19/casedistribution/csv/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  691k  100  691k    0     0   553k      0  0:00:01  0:00:01 --:--:--  553k


In [2]:
import pandas as pd
import numpy as np
import pymc3 as pm
import theano
import theano.tensor as tt

from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

In [3]:
filename = 'euData0420.csv'
dataTotal = pd.read_csv(filename,
                     usecols=['dateRep', 'countriesAndTerritories', 'cases'],
                     parse_dates=['dateRep'],infer_datetime_format=True,
                     index_col=['countriesAndTerritories', 'dateRep'],
                     squeeze=True).sort_index()
dataTotal.index.rename(names=['country', 'date'], inplace=True)

In [51]:
smoothed = pd.read_pickle('smoothed_france')

In [61]:
smoothed

date
2020-02-27      12.0
2020-02-28      18.0
2020-02-29      23.0
2020-03-01      30.0
2020-03-02      42.0
2020-03-03      59.0
2020-03-04      73.0
2020-03-05     106.0
2020-03-06     138.0
2020-03-07     179.0
2020-03-08     229.0
2020-03-09     289.0
2020-03-10     362.0
2020-03-11     441.0
2020-03-12     528.0
2020-03-13     640.0
2020-03-14     736.0
2020-03-15     857.0
2020-03-16    1003.0
2020-03-17    1138.0
2020-03-18    1282.0
2020-03-19    1399.0
2020-03-20    1641.0
2020-03-21    1837.0
2020-03-22    2053.0
2020-03-23    2341.0
2020-03-24    2626.0
2020-03-25    2945.0
2020-03-26    3163.0
2020-03-27    3415.0
2020-03-28    3851.0
2020-03-29    4089.0
2020-03-30    4231.0
2020-03-31    4434.0
2020-04-01    4500.0
2020-04-02    4374.0
2020-04-03    4230.0
2020-04-04    4133.0
2020-04-05    3947.0
2020-04-06    3710.0
2020-04-07    3708.0
2020-04-08    3788.0
2020-04-09    3609.0
2020-04-10    3493.0
2020-04-11    3603.0
2020-04-12    3468.0
2020-04-13    3346.0
2020-04-

In [53]:
country = 'France'
dataEx = dataTotal.xs(country)

In [54]:
dataEx = smoothed

In [55]:
# Gamma is 1/serial interval
# https://wwwnc.cdc.gov/eid/article/26/7/20-0282_article
# https://www.nejm.org/doi/full/10.1056/NEJMoa2001316

sigma=0.25 # some estimation for Gaussian increments, we want Laplace (see Systrom)
GAMMA = 1/7
MAX = 12.0

## Model:

$k_t$ - the number of new cases in time $t$

$R_t$ - the reproduction rate in time $t$

$\gamma$ - serial interval - fixed in Systrom's model, Wikipedia states it is somewhere in $[4,8]$

We assume that $R_t$ is a Markov process with Laplace increments of mean $0$ and standard deviation $\sigma$.
The distribution of $k_t$ is Poisson with parameter $\lambda_t$ which satisfies the equation
$\lambda_t = k_{t-1} \cdot e^{\gamma (R_t - 1)}$.

We may summarize it:

$$
\lambda_t = k_{t-1} \cdot e^{\gamma (R_t - 1)}, \quad \quad k_t = {\rm Poiss}(\lambda_t), \quad \quad P(R_t | R_{t-1}) = {\rm Laplace}(0,\sigma)
$$

In [56]:
N = dataEx.size
k = dataEx.to_numpy()[:N]

with pm.Model() as modelTest:
    R = np.empty(N,dtype=object)
    K = np.empty(N,dtype=object)
    R[0] = pm.Gamma('R_0', alpha=4, beta=1)
    K[0] = k[0]
    for t in range(1,N):
        R[t] = pm.Deterministic('R_{}'.format(t),R[t-1] + pm.Normal("Diff_{}".format(t),0,sigma))
        # par_t = pm.Deterministic('par_{}'.format(t),GAMMA*(R[t-1] - 1)) 
        # lambda_t = pm.Deterministic('lambda_{}'.format(t),K[t-1]*tt.exp(par_t))
        # Faster that way I guess, PyMC3 does not keep track of the auxiliary variables
        lambda_t = K[t-1]*tt.exp(GAMMA*(R[t-1] - 1))
        K[t] = pm.Poisson('k_{}'.format(t), lambda_t, observed=k[t])
    
with modelTest:
    trace = pm.sample(draws=1000, tune=2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Diff_54, Diff_53, Diff_52, Diff_51, Diff_50, Diff_49, Diff_48, Diff_47, Diff_46, Diff_45, Diff_44, Diff_43, Diff_42, Diff_41, Diff_40, Diff_39, Diff_38, Diff_37, Diff_36, Diff_35, Diff_34, Diff_33, Diff_32, Diff_31, Diff_30, Diff_29, Diff_28, Diff_27, Diff_26, Diff_25, Diff_24, Diff_23, Diff_22, Diff_21, Diff_20, Diff_19, Diff_18, Diff_17, Diff_16, Diff_15, Diff_14, Diff_13, Diff_12, Diff_11, Diff_10, Diff_9, Diff_8, Diff_7, Diff_6, Diff_5, Diff_4, Diff_3, Diff_2, Diff_1, R_0]
Sampling 4 chains, 0 divergences: 100%|██████████| 12000/12000 [18:36<00:00, 10.75draws/s]


In [57]:
def take_most_likely(a):
    hist_counts,hist_vals = np.histogram(a, bins=100)
    return hist_vals[hist_counts.argmax()]

In [58]:
p = 0.1

resLow, resHigh, ml = [], [], []

for t in range(N):
    val = np.sort(trace['R_{}'.format(t)])
    LEN = len(val)
    low, hi = int(p*LEN), int((1-p)*LEN)
    # print(f'Low|Hi quantiles {t}: {val[low],val[hi]}')
    resLow.append(val[low])
    resHigh.append(val[hi])
    ml.append(take_most_likely(val))

resLow = np.array(resLow)
resHigh = np.array(resHigh)
res = pd.DataFrame(index=dataEx.index, data = {'Low_90': resLow, 'High_90': resHigh, 'ML': ml})

In [59]:
res

,Low_90,High_90,ML
date,,,
2020-02-27,2.398826,3.698603,3.025561
2020-02-28,2.431120,3.624032,3.095902
2020-02-29,2.474328,3.558240,3.021949
2020-03-01,2.530011,3.499814,2.925539
2020-03-02,2.541681,3.443739,2.957557
2020-03-03,2.528527,3.362808,3.019315
2020-03-04,2.566451,3.326757,2.876124
2020-03-05,2.519564,3.216834,2.807199
2020-03-06,2.474652,3.106205,2.744519


In [60]:
res.to_pickle('pymc')